In [1]:
# import the packages
import pandas as pd
import numpy as np
import random


In [29]:
# import the data from the folder
monday = pd.read_csv('../../data/monday.csv', parse_dates=True,sep = ';')
tuesday = pd.read_csv('../../data/tuesday.csv',  parse_dates=True,sep = ';')
wednesday = pd.read_csv('../../data/wednesday.csv',  parse_dates=True,sep = ';')
thursday = pd.read_csv('../../data/thursday.csv',  parse_dates=True,sep = ';')
friday = pd.read_csv('../../data/friday.csv',  parse_dates=True,sep = ';')
monday.head()

,timestamp,customer_no,location
0,2019-09-02 07:03:00,1,dairy
1,2019-09-02 07:03:00,2,dairy
2,2019-09-02 07:04:00,3,dairy
3,2019-09-02 07:04:00,4,dairy
4,2019-09-02 07:04:00,5,spices


In [30]:
# inspect the data
monday['customer_no']

0          1
1          2
2          3
3          4
4          5
        ... 
4879    1442
4880    1444
4881    1445
4882    1446
4883    1447
Name: customer_no, Length: 4884, dtype: int64

In [40]:
# each day specifies customers with munber. We need to identify each one separately

monday['customer_id'] = 'm-'+(monday['customer_no']).astype(str)
tuesday['customer_id'] = 't-'+(tuesday['customer_no']).astype(str)
wednesday['customer_id'] = 'w-'+(wednesday['customer_no']).astype(str)
thursday['customer_id'] = 'th-'+(thursday['customer_no']).astype(str)
friday['customer_id'] = 'f-'+(friday['customer_no']).astype(str)

In [46]:
# concatenate the matrices
sm1 = monday.append(tuesday, ignore_index = True)
sm2 = sm1.append(wednesday, ignore_index = True)
sm3 = sm2.append(thursday, ignore_index = True)
sm = sm3.append(friday, ignore_index = True)


In [71]:
# sort the matrix by customer_id and timestamp
sort_sm = sm.sort_values(by=['customer_id','timestamp'])
sort_sm =sort_sm.drop(columns = ['customer_no'])
sort_sm

,timestamp,location,customer_id,location_next
19752,2019-09-06 07:00:00,dairy,f-1,drinks
19764,2019-09-06 07:04:00,spices,f-1,dairy
19767,2019-09-06 07:05:00,checkout,f-1,drinks
19771,2019-09-06 07:06:00,fruit,f-10,fruit
19792,2019-09-06 07:11:00,checkout,f-10,checkout
...,...,...,...,...
12856,2019-09-04 16:53:00,dairy,w-998,fruit
12884,2019-09-04 16:57:00,fruit,w-998,drinks
12892,2019-09-04 16:59:00,checkout,w-998,spices
12857,2019-09-04 16:53:00,fruit,w-999,spices


In [60]:
sort_sm['location_next'] = sort_sm.location.shift(-1) 

In [73]:
sort_sm.head()

,timestamp,location,customer_id,location_next
19752,2019-09-06 07:00:00,dairy,f-1,drinks
19764,2019-09-06 07:04:00,spices,f-1,dairy
19767,2019-09-06 07:05:00,checkout,f-1,drinks
19771,2019-09-06 07:06:00,fruit,f-10,fruit
19792,2019-09-06 07:11:00,checkout,f-10,checkout


In [61]:
sort_sm.location.unique()


array(['dairy', 'spices', 'fruit', 'checkout', 'drinks'], dtype=object)

In [63]:
# notes about normalize use 
# If passed ‘index’ will normalize over each row.
# default is False , which will give you the count

P = pd.crosstab(
    sort_sm['location'], 
    sort_sm['location_next'], normalize='index')
P

location_next,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,0.000000,0.287487,0.153317,0.377697,0.181499
dairy,0.390041,0.001069,0.222697,0.190212,0.195982
drinks,0.536748,0.027401,0.000000,0.219206,0.216645
fruit,0.498829,0.237993,0.137056,0.000390,0.125732
spices,0.250666,0.323655,0.272776,0.152637,0.000266


In [113]:
type(P)
weigh = P.values

In [118]:
states = sort_sm.location.unique()
state= ['checkout', 'dairy','drinks','fruit','spices']


In [122]:
probs['checkout']

[0.0,
 0.28748651564185546,
 0.15331715210355987,
 0.37769687162891047,
 0.1814994606256742]

In [116]:
probs = P.to_dict(orient='index')
# convert internal values, which is a dict to a list
for key in probs.keys():
    probs[key] = list(probs[key].values())

In [133]:
# to get the actual string representing the state, you then need to do the following 
result = []
for department in state:

    result.append(random.choices(states, weights=probs[department]))
result



[['fruit'], ['dairy'], ['dairy'], ['checkout'], ['spices']]

# EDA

- Calculate the total number of customers in each section

In [74]:
sort_sm.head()

,timestamp,location,customer_id,location_next
19752,2019-09-06 07:00:00,dairy,f-1,drinks
19764,2019-09-06 07:04:00,spices,f-1,dairy
19767,2019-09-06 07:05:00,checkout,f-1,drinks
19771,2019-09-06 07:06:00,fruit,f-10,fruit
19792,2019-09-06 07:11:00,checkout,f-10,checkout


In [89]:
# Total number of customers in each section for all week and independent of the times being there per customer 
sort_sm.location_next
sort_sm.groupby("location_next").count()

,timestamp,location,customer_id
location_next,,,
checkout,7417,7417,7417
dairy,4678,4678,4678
drinks,3905,3905,3905
fruit,5122,5122,5122
spices,3754,3754,3754


- Calculate the total number of customers in each section over time

In [ ]:
sort_sm.groupby('timestamp","location_next").count()

- make a function that 